In [0]:
import time

from databricks.sdk import WorkspaceClient
from databricks.sdk.service import sql
from pyspark.sql import SparkSession

from hotel_reservation.config import ProjectConfig

# Load configuration
config = ProjectConfig.from_yaml(config_path="../project_config.yml", env="dev")
spark = SparkSession.builder.getOrCreate()

w = WorkspaceClient()

srcs = w.data_sources.list()


alert_query = f"""
SELECT 
  (COUNT(CASE WHEN mean_absolute_error > 70000 THEN 1 END) * 100.0 / COUNT(CASE WHEN mean_absolute_error IS NOT NULL AND NOT isnan(mean_absolute_error) THEN 1 END)) AS percentage_higher_than_70000
FROM {config.catalog_name}.{config.schema_name}.model_monitoring_profile_metrics"""


query = w.queries.create(query=sql.CreateQueryRequestQuery(display_name=f'hotel-reservation-alert-query-{time.time_ns()}',
                                                           warehouse_id=srcs[0].warehouse_id,
                                                           description="Alert on hotel reservation model MAE",
                                                           query_text=alert_query))

alert = w.alerts.create(
    alert=sql.CreateAlertRequestAlert(condition=sql.AlertCondition(operand=sql.AlertConditionOperand(
        column=sql.AlertOperandColumn(name="percentage_higher_than_70000")),
            op=sql.AlertOperator.GREATER_THAN,
            threshold=sql.AlertConditionThreshold(
                value=sql.AlertOperandValue(
                    double_value=45))),
            display_name=f'hotel-reservation-mae-alert-{time.time_ns()}',
            query_id=query.id
        )
    )


# cleanup
w.queries.delete(id=query.id)
w.alerts.delete(id=alert.id)